# Predicting movie sales from Metacritic data

## 0. Business problem

The movie industry faces high financial-performance risks because of increasingly high movie-making and marketing costs and a high degree of uncertainty about audience reactions (Escoffier et al., 2015).
While Metacritic provides rich information about movies such as the critic scores, user scores, review texts and metadata, it is 
unclear how well these features can predict the monetary success. That's why this project uses historical data from Metacritic and movie sales information information to build several machine learning models that forecast whether a movie will result in low, medium or high sales. Furthermore, this project will focus on explaining which features drive these predictions. 
The final goal is to provide the movie publisher with valuable information on where to spend the marketing budget.

**Business Question** How can we predict box office perfomance of the movie to accuratly allocate marketing budget?

**Source**:
Escoffier, N., & McKelvey, B. (2015). The Wisdom of Crowds in the Movie Industry: Towards New Solutions to Reduce Uncertainties. International Journal of Arts Management, 17(2), 52–63. http://www.jstor.org/stable/24587073

### 0.1 Main research question & subquestions
**Main research question**:
How accurately can we predict a movie's box-office sales using Metacritic ratings, metadata, review texts, with particular focus on identifying the most influential predictive factors?

**Subquestions**
1. How are critic scores, user scores, genres, platforms, and release years related to the sales tiers of movies?
2. How well can different machine learning models predict the sales tier of a movie, based on structured features?
3. To what extent does adding transformers of review titles and/or movie summaries improve prediction performance compared to models using only structured features? 
4. Which features are most influential in predicting high versus low sales according to SHAP?
5. Can we identify review topics and/or movie clusters (e.g., using BERTopic and clustering methods) that are particularly associated with high or low sales tiers, and do these insights reveal distinct market segments?

The movie sales prediction dataset is contained in the dataset folder in the repository. We will read the data and clean it to make it ready for analysis.

The following information is provided on the dataset variables selected to address the research questions:

This research employs a continuous numerical variable, **Worldwide Box Office**, as the response variable. This represents the total revenue generated globally (in USD).

This study reviewed the literature and used the following 10 variables as explanatory variables:

- **X1**: Metascore
  - A weighted average of critic reviews (Scale: 0 - 100).
- **X2**: Userscore
  - Average score provided by general users (Scale: 0 - 10).
- **X3**: Production Budget
  - The estimated financial cost to produce the film (USD).
- **X4**: Genre
  - Categorical variable indicating the primary classification of the movie (e.g., Action, Comedy, Drama).
  - Movies with multiple genres are processed using One-Hot Encoding.
- **X5**: Release Date
  - Used to extract the specific month and year of release to account for seasonal market trends and inflation adjustments.
- **X6**: Runtime
  - The duration of the movie in minutes.
- **X7**: Theatre Count
  - The number of theatres showing the movie during its opening weekend, serving as a proxy for distribution width.
- **X8**: MPAA Rating
  - Categorical certification defining the target audience scope:
    - G = General Audiences
    - PG = Parental Guidance Suggested
    - PG-13 = Parents Strongly Cautioned
    - R = Restricted
    - NC-17 = Adults Only
- **X9**: Movie Summary
  - The textual plot summary of the film.
  - Used to generate semantic embeddings via Transformers to capture narrative elements.
- **X10**: Review Text
  - The raw text body of expert and user reviews.
  - Used for BERTopic modeling to identify dominant discourse topics associated with sales performance.

## 1. EDA (exploratory data analysis)

In [ ]:
import numpy as np
import pandas as pd
import os
import scipy
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # to make jupyter print all outputs, not just the last one
from IPython.core.display import HTML # to pretty print pandas df and be able to copy them over (e.g. to ppt slides)
import re, hashlib

### 1.1 Basic structure of the datasets

In [ ]:
# define file paths relative to the notebook
data_folder = "datasets"

sales_path = os.path.join(data_folder, "sales.xlsx")
userreviews_path = os.path.join(data_folder, "UserReviews.xlsx")
expertreviews_path = os.path.join(data_folder, "ExpertReviews.xlsx")
meta_path = os.path.join(data_folder, "metaClean43Brightspace.xlsx")

In [ ]:
# Load the four Excel files
UserReviews_raw = pd.read_excel(userreviews_path)
ExpertReviews_raw = pd.read_excel(expertreviews_path)
sales_raw       = pd.read_excel(sales_path)
meta_raw        = pd.read_excel(meta_path)

In [ ]:
print("There are {} rows and {} columns in the user review dataset".format(UserReviews_raw.shape[0], UserReviews_raw.shape[1]))
print("There are {} rows and {} columns in the expert review dataset".format(ExpertReviews_raw.shape[0], ExpertReviews_raw.shape[1]))
print("There are {} rows and {} columns in the sales dataset".format(sales_raw.shape[0], sales_raw.shape[1]))
print("There are {} rows and {} columns in the meta dataset".format(meta_raw.shape[0], meta_raw.shape[1]))

In [ ]:
print("UserReview ")
UserReviews_raw.head()
UserReviews_raw.tail()
print("Expert Review")
ExpertReviews_raw.head()
ExpertReviews_raw.tail()
print("Meta data")
meta_raw.head()
meta_raw.tail()
print("Sales data")
sales_raw.head()
sales_raw.tail()


In [ ]:
def print_basic_info(name, df):
    print(f"\n{name}")
    print("-" * len(name))
    print("Columns:", list(df.columns))
    print("\nMissing values per column:")
    display(df.isna().sum().sort_values(ascending=False).head(15))

print_basic_info("UserReviews_raw", UserReviews_raw)
print_basic_info("ExpertReviews_raw", ExpertReviews_raw)
print_basic_info("sales_raw", sales_raw)
print_basic_info("meta_raw", meta_raw)

In [ ]:
def check_mixed_types(df, df_name):
    """
    Check for columns with mixed data types in a DataFrame.
    
    Args:
        df: The DataFrame to check
        df_name: Name of the DataFrame (for display purposes)
    """
    print(f"\n{'='*50}")
    print(f"Checking: {df_name}")
    print('='*50)
    
    found_mixed = False
    
    for col in df.columns:
        
        # Map type names to more readable versions
        # NaN values show as 'float', so we relabel them as 'NaN/missing'
        def get_type(x):
            if pd.isna(x):
                return 'NaN/missing'
            return type(x).__name__
        
        types = df[col].apply(get_type).value_counts().to_dict()
        
        # Only flag columns with actual mixed types (ignoring NaN)
        non_nan_types = {k: v for k, v in types.items() if k != 'NaN/missing'}
        
        if len(non_nan_types) > 1:
            found_mixed = True
            type_parts = [f"{count} {dtype}" for dtype, count in types.items()]
            print(f"  '{col}' has mixed types: {', '.join(type_parts)}")
    
    if not found_mixed:
        print("  No mixed types found - all columns are clean!")

check_mixed_types(UserReviews_raw, "UserReviews_raw")
check_mixed_types(ExpertReviews_raw, "ExpertReviews_raw")
check_mixed_types(sales_raw, "sales_raw")
check_mixed_types(meta_raw, "meta_raw")

### Reflection on the raw datasets

Based on the first EDA and the 10 variables (X1-X10) identified for this research, we can see which problems exist in each table and what cleaning steps are needed.

#### Required Variables Overview

Our research focuses on 10 specific variables:
- **X1**: Metascore (from `meta_raw`)  
- **X2**: Userscore (from `meta_raw`)  
- **X3**: Production Budget (from `sales_raw`)  
- **X4**: Genre (from `meta_raw`)  
- **X5**: Release Date (from `meta_raw` or `sales_raw`)  
- **X6**: Runtime (from `meta_raw`)  
- **X7**: Theatre Count (from `sales_raw`)  
- **X8**: MPAA Rating (from `meta_raw`)  
- **X9**: Movie Summary (from `meta_raw`)  
- **X10**: Review Text (from `UserReviews_raw` and `ExpertReviews_raw`)  
- **Target**: Worldwide Box Office (from `sales_raw`)

#### meta_raw

- Shape: 11,364 rows and 13 columns. This is our primary movie-level metadata table.  
- **Critical columns for X1-X2, X4, X5-X6, X8-X9**:  
  - `metascore` (X1): No missing values, already numeric. This is our primary critic rating.  
  - `userscore` (X2): Contains some missing values, needs numeric conversion.  
  - `genre` (X4): Contains genre information, will need One-Hot Encoding for multiple genres.  
  - `RelDate` (X5): Already in clean date format, no missing values. Will extract month and year.  
  - `runtime` (X6): Contains movie duration in minutes, needs numeric cleaning.  
  - `rating` (X8): MPAA rating (G, PG, PG-13, R, NC-17), categorical variable.  
  - `summary` (X9): Plot summary text, many missing values but essential for semantic embeddings.  
  - `url`: No missing values, serves as our primary linking key.

Cleaning implications:
- Standardise `RelDate` into a single `releasedate` column using `standardize_meta_dates`.  
- Ensure `metascore`, `userscore` and `runtime` are properly numeric.  
- Keep `summary` text for later transformer-based embeddings, noting missing values.  
- Process `genre` for One-Hot Encoding.  
- Validate `rating` contains only valid MPAA categories.

#### sales_raw

- Shape: 30,612 rows and 16 columns. This contains our target variable and X3, X5, X7.  
- **Critical columns for X3, X5, X7 and target**:  
  - `worldwide_box_office`: Our target variable, comes as text and needs numeric conversion.  
  - `production_budget` (X3): Many missing values (~20,000), needs numeric conversion from text.  
  - `theatre_count` (X7): Distribution width proxy, many missing values, needs numeric conversion.  
  - `release_date` (X5): Free-form text date needs conversion to datetime for seasonal trends.  
  - `url`: No missing values, used for linking to meta table.  
- Column `Unnamed: 8` is completely empty and should be dropped.

Cleaning implications:
- Convert `worldwide_box_office`, `production_budget` and `theatre_count` to numeric using `get_numeric_value`.  
- Convert `release_date` to datetime via `clean_sales_dates`, with fallback on `year` where needed.  
- Handle missing values in budget and theatre count appropriately for modelling.  
- Drop irrelevant columns like `Unnamed: 8`.

#### UserReviews_raw & ExpertReviews_raw

- Combined shapes: 319,662 user reviews (7 columns) + 238,973 expert reviews (5 columns).  
- **Critical column for X10**:  
  - `Rev`: Review text body, essential for BERTopic modeling and identifying discourse topics.  
  - `url`: Links reviews to specific movies for aggregation.  
  - `dateP`: Review dates, though not in our X1-X10, useful for temporal analysis.  
- User reviews have ~3,400 missing values in reviewer and date fields.  
- Expert reviews have only 2 missing values across key fields.

Cleaning implications:
- Primary focus: ensure `Rev` text is preserved and linked to correct movies via `url`.  
- Convert dates to datetime format for consistency.  
- Clean reviewer names (replace missing with "Unknown Reviewer").  
- Create review IDs for tracking individual reviews.  
- Plan to aggregate reviews at movie level or use for BERTopic analysis.

#### Across all tables

- All tables have a `url` column with no missing values, making it our natural linking key.  
- However, URLs may have slight variations, so we need standardisation to create a consistent `movie_id`.  
- After cleaning, we'll have a movie-level dataset with all X1-X10 variables aligned.

Helper functions needed:
- `clean_movie_name` and `extract_title_from_url` to standardise movie identifiers.  
- `collect_all_movie_records`, `create_movie_dimension_table` and `add_movie_ids_to_datasets` to create a movie dimension table.  
- Data type conversion helpers for numeric fields (budget, scores, runtime, theatre count).  
- Date standardisation functions for release dates across tables.

After these cleaning steps, we'll have:
- A clean movie-level dataset with all 10 predictor variables (X1-X10) and the target (Worldwide Box Office).  
- Consistent date formats for temporal feature extraction (month, year).  
- Properly formatted numeric and categorical variables ready for One-Hot Encoding.  
- Review texts prepared for transformer embeddings and topic modeling.  
- All tables linkable via `movie_id` for seamless joins.


### 1.2 Data Cleaning Implementation

Now we implement the cleaning steps outlined in the reflection above. We'll create reusable helper functions following best practices from the tutorials.

#### 1: Extracting numeric values from strings

In [ ]:
# Numeric conversion helper function
def get_numeric_value(value):
    # Handle missing values and text placeholders
    if pd.isna(value) or (isinstance(value, str) and value.strip().lower() in ['', 'n/a', 'na', 'none', 'tbd']):
        return np.nan
    # Already numeric - just convert to float
    if isinstance(value, (int, float)):
        return float(value)
    # Remove currency symbols, commas, spaces, percentages and convert
    try:
        return float(re.sub(r'[$,€£¥\s%]', '', str(value).strip()))
    except ValueError:
        return np.nan

# Apply to meta dataset (X1, X2, X6)
meta_clean = meta_raw.copy()
meta_clean[['metascore', 'userscore', 'runtime']] = meta_clean[['metascore', 'userscore', 'runtime']].applymap(get_numeric_value)

# Apply to sales dataset (target, X3, X7)
sales_clean = sales_raw.drop(columns=['Unnamed: 8'], errors='ignore')
sales_clean[['worldwide_box_office', 'production_budget', 'theatre_count']] = sales_clean[['worldwide_box_office', 'production_budget', 'theatre_count']].applymap(get_numeric_value)

# Apply to user reviews (not used in X1-X10 but cleaned for consistency)
UserReviews_clean = UserReviews_raw.copy()
UserReviews_clean['idvscore'] = UserReviews_clean['idvscore'].apply(get_numeric_value)

# Apply to expert reviews (not used in X1-X10 but cleaned for consistency)
ExpertReviews_clean = ExpertReviews_raw.copy()
ExpertReviews_clean['idvscore'] = ExpertReviews_clean['idvscore'].apply(get_numeric_value)

print(f"✓ Numeric cleaning complete: meta({len(meta_clean)}), sales({len(sales_clean)}), user({len(UserReviews_clean)}), expert({len(ExpertReviews_clean)})")

# Verification
print(f"Types: metascore={meta_clean['metascore'].dtype}, worldwide_box_office={sales_clean['worldwide_box_office'].dtype}")
print(f"Ranges: metascore[{meta_clean['metascore'].min():.0f}-{meta_clean['metascore'].max():.0f}], box_office[${sales_clean['worldwide_box_office'].min():,.0f}-${sales_clean['worldwide_box_office'].max():,.0f}]")
print(f"Missing: metascore={meta_clean['metascore'].isna().sum()}, production_budget={sales_clean['production_budget'].isna().sum()}, theatre_count={sales_clean['theatre_count'].isna().sum()}")

#### 2: Date conversion

In [ ]:
# Date conversion
def parse_date_vectorized(date_series, fallback_year_series=None):
    # Step 1: Remove ordinal suffixes from all dates at once (e.g., "January 1st" -> "January 1")
    cleaned = date_series.astype(str).str.replace(r'(\d+)(st|nd|rd|th)', r'\1', regex=True)
    
    # Step 2: Add fallback year to dates without a 4-digit year
    if fallback_year_series is not None:
        mask = ~cleaned.str.contains(r'\d{4}', na=False)  # Find dates without year
        cleaned = cleaned.where(~mask, cleaned + ', ' + fallback_year_series.astype(str))
    
    # Step 3: Convert to datetime, invalid dates become NaT
    return pd.to_datetime(cleaned, errors='coerce')

# Apply to meta dataset (X5) - RelDate already in clean format, just convert
meta_clean['releasedate'] = pd.to_datetime(meta_clean['RelDate'], errors='coerce')

# Apply to sales dataset (X5) - use 'year' column as fallback for incomplete dates
sales_clean['releasedate'] = parse_date_vectorized(sales_clean['release_date'], sales_clean['year'])

# Apply to review datasets - already in standard format, direct conversion
UserReviews_clean['dateP'] = pd.to_datetime(UserReviews_clean['dateP'], errors='coerce')
ExpertReviews_clean['dateP'] = pd.to_datetime(ExpertReviews_clean['dateP'], errors='coerce')

print(f" Date cleaning complete: all datasets have standardized datetime columns")

# Verification: check date ranges and missing values
print(f" Date ranges: meta[{meta_clean['releasedate'].min()} to {meta_clean['releasedate'].max()}], sales[{sales_clean['releasedate'].min()} to {sales_clean['releasedate'].max()}]")
print(f" Missing dates: meta={meta_clean['releasedate'].isna().sum()}, sales={sales_clean['releasedate'].isna().sum()}")

#### 3. Movie linking

In [ ]:
def clean_movie_name(text):
    """Make movie names consistent for matching across datasets."""
    if pd.isna(text) or text is None:
        return ""
    name = str(text).lower().strip().rstrip("/")
    name = name.rsplit("/", 1)[-1]  # take last part after slash (for URLs)
    name = name.replace("-", " ").replace("_", " ")
    # remove brackets content
    name = re.sub(r"\([^)]*\)|\[[^\]]*\]", "", name)
    # remove year at end
    name = re.sub(r"[\(\[]\s*(?:19|20)\d{2}[^)\]]*\s*[\)\]]\s*$|(?:19|20)\d{2}\s*$", "", name)
    # keep only letters, digits, spaces
    name = re.sub(r"[^a-z0-9\s\-']", "", name)
    # collapse spaces
    name = re.sub(r"\s+", " ", name).strip()
    return name

# Create cleaned names and movie_id for both datasets
meta_clean['cleaned_name'] = meta_clean['url'].apply(clean_movie_name)
meta_clean['movie_id'] = meta_clean['cleaned_name'].apply(lambda x: hashlib.md5(x.encode()).hexdigest()[:12] if x else None)

sales_clean['cleaned_name'] = sales_clean['url'].apply(clean_movie_name)
sales_clean['movie_id'] = sales_clean['cleaned_name'].apply(lambda x: hashlib.md5(x.encode()).hexdigest()[:12] if x else None)


# Check overlap
meta_ids = set(meta_clean['movie_id'].dropna())
sales_ids = set(sales_clean['movie_id'].dropna())
overlap = len(meta_ids.intersection(sales_ids))
print(f"✓ Overlapping movies: {overlap}")
print(f"  Meta unique: {len(meta_ids)}, Sales unique: {len(sales_ids)}")

# Merge datasets - using sales as main source with LEFT JOIN
# This keeps ALL movies from sales, even if they don't have metadata
movie_data = sales_clean.merge(meta_clean, on='movie_id', how='left', suffixes=('_sales', '_meta'))
print(f"✓ Merged dataset: {len(movie_data)} movies (all from sales dataset)")
print(f"  Movies with metadata: {movie_data['metascore'].notna().sum()}")
print(f"  Movies without metadata: {movie_data['metascore'].isna().sum()}")

In [ ]:
# Since sales is now the base table, we need to handle column selection differently
# Columns from sales (base) don't have suffixes
# Columns from meta have _meta suffix
# For overlapping columns (genre, runtime, releasedate), prefer meta version when available, otherwise use sales

# First, let's create consolidated columns for the overlapping fields
# For genre: prefer meta (more complete), fallback to sales
movie_data['genre'] = movie_data['genre_meta'].fillna(movie_data['genre_sales'])

# For releasedate: prefer sales (already cleaned), fallback to meta
movie_data['releasedate'] = movie_data['releasedate_sales'].fillna(movie_data['releasedate_meta'])

# For runtime: prefer meta (more accurate), fallback to sales
movie_data['runtime'] = movie_data['runtime_meta'].fillna(movie_data['runtime_sales'])

# Now select final columns
movie_data = movie_data[[
    'movie_id',
    'metascore',              # X1 (from meta)
    'userscore',              # X2 (from meta)
    'production_budget',      # X3 (from sales, no suffix)
    'genre',                  # X4 (consolidated)
    'releasedate',            # X5 (consolidated)
    'runtime',                # X6 (consolidated)
    'theatre_count',          # X7 (from sales, no suffix)
    'rating',                 # X8 (from meta)
    'summary',                # X9 (from meta)
    'worldwide_box_office'    # Target (from sales, no suffix)
    # X10 (review text) will be aggregated separately
]]

# Remove movies with missing movie_id (mostly year-only titles that can't be matched)
print(f"Removing {movie_data['movie_id'].isna().sum()} movies with missing movie_id...")
movie_data = movie_data[movie_data['movie_id'].notna()].copy()

print(f"✓ Movie-level dataset: {len(movie_data)} movies with X1-X9 + target")
print(f"\n✓ Missing values:")
print(movie_data.isnull().sum())
print(f"\n✓ Complete cases (no missing values): {movie_data.dropna().shape[0]} movies")


In [ ]:
# Filter to only movies with box office data (target variable)
movie_data_with_target = movie_data[movie_data['worldwide_box_office'].notna()].copy()
print(f"✓ Movies with box office data: {len(movie_data_with_target)}")

# Now add movie_id to review datasets
UserReviews_clean['cleaned_name'] = UserReviews_clean['url'].apply(clean_movie_name)
UserReviews_clean['movie_id'] = UserReviews_clean['cleaned_name'].apply(lambda x: hashlib.md5(x.encode()).hexdigest()[:12] if x else None)

ExpertReviews_clean['cleaned_name'] = ExpertReviews_clean['url'].apply(clean_movie_name)
ExpertReviews_clean['movie_id'] = ExpertReviews_clean['cleaned_name'].apply(lambda x: hashlib.md5(x.encode()).hexdigest()[:12] if x else None)

print(f"✓ Added movie_id to reviews")

# Aggregate review texts
user_reviews_agg = UserReviews_clean.groupby('movie_id')['Rev'].apply(lambda x: ' '.join(x.dropna().astype(str))).reset_index()
user_reviews_agg.columns = ['movie_id', 'user_review_text']

expert_reviews_agg = ExpertReviews_clean.groupby('movie_id')['Rev'].apply(lambda x: ' '.join(x.dropna().astype(str))).reset_index()
expert_reviews_agg.columns = ['movie_id', 'expert_review_text']

# Merge everything
movie_data_complete = movie_data_with_target.merge(user_reviews_agg, on='movie_id', how='left')
movie_data_complete = movie_data_complete.merge(expert_reviews_agg, on='movie_id', how='left')

movie_data_complete['review_text'] = (
    movie_data_complete['user_review_text'].fillna('') + ' ' + 
    movie_data_complete['expert_review_text'].fillna('')
).str.strip()

movie_data_complete['release_month'] = movie_data_complete['releasedate'].dt.month
movie_data_complete['release_year'] = movie_data_complete['releasedate'].dt.year

print(f"✓ Final dataset: {len(movie_data_complete)} movies with all X1-X10 variables")

In [ ]:
movie_data.head()

In [ ]:
movie_data_complete.head()

In [ ]:
# again show the information on the the movie_data_complete dataframe

print(f"✓ Movie-level dataset: {len(movie_data_complete)} movies with X1-X9 + target")
print(f"\n✓ Missing values:")
print(movie_data_complete.isnull().sum())
print(f"\n✓ Complete cases (no missing values): {movie_data_complete.dropna().shape[0]} movies")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt #for creating more graphical illustrations

In [ ]:
#Create a fresh dataframe so the movie data is not fragmented and we dont get that warning
movie_data_complete = movie_data_complete.copy()

In [ ]:
def EDA_plots(df, column, target='worldwide_box_office'):
    """
    EDA for a single numeric feature (because this is NOT a classifaction problem, but a (linear) regression problem).

    Plots:
    1. Histogram of the feature
    2. Boxplot of the feature
    3. KDE distribution of the feature
    4. Scatterplot: feature vs target (e.g. worldwide_box_office)

    Parameters:
    df      : DataFrame
    column  : Name of the feature to analyze
    target  : Target column (default: 'worldwide_box_office')
    """

    # Skip non-numeric columns to avoid seaborn errors
    #if not pd.api.types.is_numeric_dtype(df[column]):
     #   print(f"Skipping '{column}': not numeric")
     #   return

    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    plt.suptitle(f"EDA for: {column}", fontsize=16)

    # 1. Histogram
    sns.histplot(df[column], bins=30, ax=axes[0, 0])
    axes[0, 0].set_title("Histogram")

    # 2. Boxplot
    sns.boxplot(x=df[column], ax=axes[0, 1])
    axes[0, 1].set_title("Boxplot")

    # 3. KDE distribution
    sns.kdeplot(df[column], ax=axes[1, 0])
    axes[1, 0].set_title("KDE Distribution")

    # 4. Relationship with target (scatterplot)
    if target in df.columns and pd.api.types.is_numeric_dtype(df[target]):
        sns.scatterplot(
            x=df[column],
            y=df[target],
            ax=axes[1, 1],
            s=10,
            alpha=0.4
        )
        axes[1, 1].set_title(f"{column} vs {target}")
    else:
        axes[1, 1].axis("off")
        axes[1, 1].set_title("No numeric target column found")

    plt.tight_layout()
    plt.show()


In [ ]:
#create a dataframe with a placeholder for all the created column in the customer_final dataframe. 
df_EDA_features= [
    'metascore',
    'userscore',
    'production_budget',
    'release_month',
    'release_year',
    'runtime',
    'theatre_count',
    'worldwide_box_office'       
]

for col in df_EDA_features:
    EDA_plots(movie_data_complete, col, target='worldwide_box_office')


### Reflection on the EDA plots per feature

- **Metascore**: This feature shows an almost normal distribution, centered around 50-70. There is only one outlier and the scatterplot shows no strong linear relationship between the worldwide box office and metascore. This means that the score of the critics alone is NOT a strong predictor of the box office performance

- **Userscore**: This feature shows a right skewed distribution, with most movies receiving an 5 - 8. The boxplot shows some outliers around the 1, these could be people that did not rate the movie honestly. The scatterplot shows that popular movies can have high box office results, but most of them do not. Meaning that userscore alone is also not a strong predictor of box office performance. 

- **Production_budget**: This features showes a heavily left skewed distribution in which most movies have a production budget of less then 50000000. However, the boxplot showes that there are also movies that have budgets as high as 400,000,000. Interestingly, the scatterplot does show a positive linear relationship. Although not every movie with high production budget succeeds, most of them do! 

- **Release_month**: The release month is evenly distributed, with some months showing small dips. The scatterplots show some small peaks in the summer and winter. It is not a very strong predictor by itself, but could be useful after one hot encoding in the future.

- **Release_year**: The released year show very expected behaviour. More movies have been released in more recent years and the movies tend to earn more in recent years. This is probably due to inflation and general market growth.

- **Runtime**: The distribution shows that most movies have a runtime of around 100-150 minutes. There is no clear relationship between runtime and box office performance based on the scatterplot. This is a weak predictor by itself.

- **Theatre_count**: This distribution is extremely skewed. Most movies open in very few theatres, but a few movies open in way more theatres. These will probably be blockbuster movies. The scatterplot however shows a very strong positive relationship between the amount of theatres and the worldwide box office. It is also seen that as soon as a movie is showed in more then 4000 theatres, it almost always means more revenue. This feature might need a log transform to handle the skeweness but it will be a very strong predictor.

- **Worldwide_box_office**: The target shows a heaviliy left skewed distribution. Meaning that most movies make a few million and some movies make billions. 


### 1.4 Inferential statistics
In this chapter, we will move from descriptive statistics to inferential statistics. The numerical features (X1, X2, X3, X6, X7) will be examined with Pearson and Spearmon correlation testing. The categorical features (X4, X5, X8) will be examined with either ANOVA or Kruskal wallis depending on the characterics. 

### Assummptions reflection: 
- Pearson correlation assumptions:
    1. The relationship between the variables is linear
    2. The data is normaly distributed
    3. The data is on a continuous scale
    4. There should be no extreme outliers 

- **X1**: Metascore does not violate any assumptions. Although the scatterplot does not show strong linearity, is does show some. 
- **X2**: Userscore **does** violate assumptions. There are some outliers at the 1, and the distribution is right skewed. Therefore, this is not a good fit for the pearson   correlation test
- **X3**: Altough production budget shows linearity, there are extreme outliers and the data is highly left skewed. Therefore it **does** violate assumptions
- **X4**: Genre is not a numerical feature, does not fit pearson
- **X5**: Release date is not a numerical feature, does not fit pearson 
- **X6**: Runtime shows a weak linearity and a right skewed distribution and **does** violate the assumptions
- **X7**: Theatre count **does** violate the assumption due to the extreme skew.
- **X8**: MPAA Rating is not a numerical feature, does not fit pearson
- **X9**: Movie Summary is not a numerical feature, does not fit pearson
- **X10**: Review Text is not a numerical feature, does not fit pearson

Conclusion: the only feature suited for pearson correlation is metascore. However, the target feature violates the assumptions as well. Since pearson correlation is a bivariate test, Pearson is not suited at all. To find correlation but mitigate these violations, Spearman rank correlation is used.



#### 1.4.1: Correlation analysis


In [ ]:
# 1. Correlation analysis for continuous data
# Based on our assumptions reflection, we use Spearman correlation due to:
# - Non-normal distributions (skewed data)
# - Presence of outliers
# - Non-linear relationships

# Select all continuous features and the target variable
scale_features = [
    'metascore',              # X1
    'userscore',              # X2
    'production_budget',      # X3
    'runtime',                # X6
    'theatre_count',          # X7
    'release_year',           # from X5
    'worldwide_box_office'    # Target
]

# Calculate Spearman correlation (rank-based, robust to outliers and non-linearity)
spearman_corr = movie_data_complete[scale_features].corr(method='spearman')

print("Spearman Correlation Matrix:")
print("=" * 80)
print(spearman_corr.round(3))
print("\n")

# Focus on correlations with target variable
print("Spearman Correlations with Worldwide Box Office:")
print("=" * 80)
target_corr = spearman_corr['worldwide_box_office'].sort_values(ascending=False)
print(target_corr)

# Visualize with heatmaps
fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# Spearman correlation heatmap
sns.heatmap(
    spearman_corr, 
    annot=True, 
    cmap='coolwarm', 
    fmt=".2f", 
    linewidths=0.5,
    center=0,
    vmin=-1,
    vmax=1,
    ax=axes[0],
    cbar_kws={'label': 'Correlation Coefficient'}
)
axes[0].set_title('Spearman Correlation Matrix\n(Rank-based, robust to outliers)', fontsize=14, pad=20)

# For comparison, also show Pearson (even though assumptions are violated)
pearson_corr = movie_data_complete[scale_features].corr(method='pearson')
sns.heatmap(
    pearson_corr, 
    annot=True, 
    cmap='coolwarm', 
    fmt=".2f", 
    linewidths=0.5,
    center=0,
    vmin=-1,
    vmax=1,
    ax=axes[1],
    cbar_kws={'label': 'Correlation Coefficient'}
)
axes[1].set_title('Pearson Correlation Matrix\n(For comparison - assumptions violated)', fontsize=14, pad=20)

plt.tight_layout()
plt.show()

# Statistical significance testing for key correlations with target
print("\n" + "=" * 80)
print("Statistical Significance Tests (Spearman) for correlations with target:")
print("=" * 80)

from scipy.stats import spearmanr

for feature in scale_features:
    if feature != 'worldwide_box_office':
        # Remove NaN values for the pair
        valid_data = movie_data_complete[[feature, 'worldwide_box_office']].dropna()
        
        if len(valid_data) > 0:
            corr, p_value = spearmanr(valid_data[feature], valid_data['worldwide_box_office'])
            significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "ns"
            print(f"{feature:25s}: ρ = {corr:6.3f}, p = {p_value:.4e} {significance}")

print("\nSignificance levels: *** p<0.001, ** p<0.01, * p<0.05, ns = not significant")

# Juliusz writes reflection here

### 1.5 Handling outliers and missing data

In this chapter, we will handle the missing values and outliers before we do any feature engineering. Beacause we have a lot of missing data (see chapter 1.2.1) we need to impute data and remove outliers. Looking at chapter 1.2.1 we see that worldwide_box_office has around 30% missing values and metascore, userscore, production_budget, theathre count, rating and summary have about 60% to as much as 85% missing values. 

1. First we want to drop the missing rows in release_month and release_year that have since they have a very small amount of missing values.Just for completeness
2. Second, we want to impute the data for the features that have some missing values but not that much, like release_date and genre. 
3. For the features with a lot of missing values, we check to what extend they might have predictive power from the correlation analysis. It could be very interesting to see whether models will have better or worse performance when keeping missing values or dropping them.
4. Lasty, we prepare the data for feature engineering and modelling by transforming the skewed data with the log transformation.

### 1.5.1 Missing values

In [ ]:
#Lets take one more look at the missing valiues
print(f"✓ Movie-level dataset: {len(movie_data_complete)} movies with X1-X9 + target")
print(f"\n✓ Missing values:")
print(movie_data_complete.isnull().sum())
print(f"\n✓ Complete cases (no missing values): {movie_data_complete.dropna().shape[0]} movies")

#### 1.5.1.1 features that have a small amount of missing data

In [ ]:
#Step one: drop the missing values from the movie_data_complete file and print the new values as a sanity check
movie_data_complete = movie_data_complete.dropna(subset=['release_month','release_year']).copy()
print(f"\n✓ Missing values:")
print(movie_data_complete[['release_month', 'release_year']].isna().sum())

#### 1.5.1.2 features with moderate amount of missing data

In [ ]:
#Now we create a new column called primary genre so we can impute te median of the runtime based on their genre
#This is basically done because the runtime has more then 2500 missing values and this way we can take the median of the runtime per genre. This will differentiate a little and makes it more precise
# 1. gives missing genres a value "unknown"
movie_data_complete['genre'] = movie_data_complete['genre'].fillna('Unknown')

# 2. Create a new column for the primary genre (this is for the imputation of runtime)
movie_data_complete['primary_genre'] = movie_data_complete['genre'].str.split(',').str[0].str.strip()

# 3. Runtime imputation using primary_genre. We also create a new column with an integer 0 or 1 to tell the model in the future which runtimes are real values and which have been imputed
movie_data_complete['runtime_missing'] = movie_data_complete['runtime'].isna().astype(int)

movie_data_complete['runtime'] = movie_data_complete.groupby('primary_genre')['runtime'] \
                 .transform(lambda x: x.fillna(x.median())) #fill the empty rows with the median runtime per pprimary genre

movie_data_complete.head(5)



#### 1.5.1.3 features that have a lot of missing data

We want to keep as much information as we can. Also, the features with the most missing data are actually the features that will very likely have the biggest predictive power. This is seen in the correlation analysis. That's why we keep the features and we again create another column with either 0 of 1 for these features telling the model if the data was already there or not. 

In [ ]:
high_missing_numeric = ['metascore', 'userscore',
                        'production_budget', 'theatre_count']

#Loop through all the high missing numerical columns to create a new column, and put the median value of that corresponding column in there. Also fopr the Nan's.
for col in high_missing_numeric: 
    # 1. Indicator whether it was missing originally
    movie_data_complete[col + '_missing'] = movie_data_complete[col].isna().astype(int)
    
    # 2. Give the columns the median value
    median_val = movie_data_complete[col].median()
    
    # 3. Fill NaNs with the median
    movie_data_complete[col] = movie_data_complete[col].fillna(median_val)

#Now give the categorical feature rating an unknown value if it is NaN
# Indicator for missing ratings
movie_data_complete['rating_missing'] = movie_data_complete['rating'].isna().astype(int)

# Impute missing ratings as 'Unknown' and create another column for the indication if it was missing or not
movie_data_complete['rating'] = movie_data_complete['rating'].fillna('Unknown')

#lastly impute 'Unknown' into the summary column and create a summary_missing column
movie_data_complete['summary_missing'] = movie_data_complete['summary'].isna().astype(int)
movie_data_complete['summary'] = movie_data_complete['summary'].fillna('Unknown')

movie_data_complete.head()

### 1.5.2 Feature transformation to get rid of the CRAZZYYY skew 

In de EDA plots, it is clearly seen that worldwide box office (the target), product budget and runtime have very skewed distributions. These get a log transformation.


In [ ]:
#log transformations on all the skewed features
movie_data_complete['worldwide_box_office_log'] = np.log1p(movie_data_complete['worldwide_box_office'])
movie_data_complete['production_budget_log']    = np.log1p(movie_data_complete['production_budget'])
movie_data_complete['theatre_count_log']        = np.log1p(movie_data_complete['theatre_count'])
movie_data_complete['runtime_log']        = np.log1p(movie_data_complete['runtime'])

movie_data_complete.head()

# Now lets reuse that STUNNING EDA plot function to see if it worked
skewed_features = [
    'worldwide_box_office_log',
    'production_budget_log',
    'theatre_count_log',
    'runtime_log'       
]

for col in skewed_features:
    EDA_plots(movie_data_complete, col, target='worldwide_box_office')

## 2. Feature Engineering

This section transforms raw variables into model-ready features. Feature engineering is critical because:
- **Tree-based models** (Random Forest, XGBoost) work best with properly encoded categorical variables
- **Multi-label features** (genres) require special handling to preserve all information
- **Temporal patterns** in release dates can capture seasonal box office trends

**Features to Engineer:**
| Variable | Type | Transformation |
|----------|------|----------------|
| Genre (X4) | Multi-label categorical | MultiLabelBinarizer → binary columns |
| MPAA Rating (X8) | Categorical | Clean + One-hot encode |
| Release Date (X5) | Date | Extract month, year, season indicators |
| Production Budget (X3) | Numeric (skewed) | Log transform |
| Box Office (Target) | Numeric (skewed) | Log transform → Tertile classification |

**Data Leakage Considerations:**
We exclude the following features from modeling as they contain information only available *after* a movie's release:
- `opening_weekend`: Directly correlated with final box office
- `theatre_count`: Determined by distributor based on expected performance
- `avg_run_per_theatre`: Post-release metric

These would "leak" the target variable into features, making the model unrealistically accurate but useless for pre-release predictions.


# categoricals vs target (genre / rating / release year)

In [ ]:
(
    movie_data_complete
    .groupby("genre", observed=False)["worldwide_box_office"]
    .agg(["count", "mean", "median"])
    .sort_values("mean", ascending=False)
    .head(10)
)


### Categorical variables and sales (genres)

We grouped the data by the full `genre` string and computed the **count**, **mean** and **median** of `worldwide_box_office` for each group.

What we see in the top rows:

- The highest average box office comes from **long genre combinations** such as  
  `Adventure,Fantasy,Comedy,Romance,Family,Musical` or  
  `Action,Adventure,Comedy,Crime,Animation,Family`.  
  These are all big, broad, family-oriented combinations (adventure, fantasy, comedy, family, animation).

- For most of these top genre combinations the **count is only 1**.  
  This means the high mean is driven by a **single blockbuster** in that category, so we cannot generalise too much from these rows.

- The only combination in the top 10 with more observations is  
  `Adventure,Mystery,Fantasy,Family` (5 movies) with an average box office of roughly **8.9**.  
  This confirms that large adventure/fantasy family movies are among the strongest commercial performers.

Implications for modelling:

- Genre clearly matters, but using the **full genre string** leads to many very rare combinations (mostly count = 1).  
- Before using genre in our models, it will be better to **simplify it**, for example by extracting a `main_genre` or grouping genres into a smaller number of high-level categories. This reduces sparsity and makes the model easier to interpret.


# outliers and log-transformation

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

sns.histplot(movie_data_complete["worldwide_box_office"].dropna(), ax=axes[0], bins=50)
axes[0].set_title("Worldwide box office (raw)")

sns.histplot(
    np.log1p(movie_data_complete["worldwide_box_office"].dropna()),
    ax=axes[1],
    bins=50
)
axes[1].set_title("Worldwide box office (log-transformed)")

plt.tight_layout()
plt.show()


### Outliers and transformations

The two histograms compare the distribution of `worldwide_box_office` **before** and **after** a log transformation.

**Raw scale (left plot)**  
- The distribution is extremely right-skewed.  
- Most movies earn relatively modest amounts, and a small number of blockbusters reach very high box office values.  
- The long tail on the right makes it hard for a model to treat “normal” movies and extreme hits in a balanced way, because the loss will be dominated by a few very large values.

**Log-transformed scale (right plot)**  
- After applying `log(1 + worldwide_box_office)`, the histogram becomes much more spread out and closer to a bell-shaped distribution.  
- The blockbusters are still at the high end, but they are no longer so extreme compared to the bulk of the data.  
- This makes patterns easier to see and typically leads to a more stable regression model.

**Implication for modelling**  
For these reasons, it is reasonable to train our models on  
`log(1 + worldwide_box_office)` instead of the raw box office.  
We can always transform predictions back to the original scale by applying the inverse transformation `exp(pred) - 1`.


# Missing Values

### Missing values: final decisions

Based on the missing value inspection:

- We drop completely empty or irrelevant columns (e.g. `Unnamed: 8`).
- **For our modelling dataset we keep only movies with a non-missing `worldwide_box_office` and `production_budget`, because both are essential for our research question.**
- For other features with moderate missingness (e.g. `runtime`), we either:
  - drop the rows (for smaller amounts of missingness), or
  - apply simple imputation (e.g. median or most frequent value) in a later preprocessing step.

These choices are based on the trade-off between data quality and keeping enough observations for training.


# 2.1 Genre Multi-Label Encoding (X4)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# Step 1: Split genres into lists
movie_data_complete['genre_list'] = movie_data_complete['genre'].apply(
    lambda x: [g.strip() for g in str(x).split(',')]
)

print(f"\n1. Found {movie_data_complete['genre'].nunique()} unique genre combinations")

# Step 2: Apply MultiLabelBinarizer
mlb = MultiLabelBinarizer()
genre_encoded = mlb.fit_transform(movie_data_complete['genre_list'])

genre_df = pd.DataFrame(
    genre_encoded,
    columns=[f'genre_{genre}' for genre in mlb.classes_],
    index=movie_data_complete.index
)

print(f"\n2. Created {len(mlb.classes_)} binary genre columns")
print(f"\nTop 5 genres:")
print(genre_df.sum().sort_values(ascending=False).head(5))

# Step 3: Merge back
movie_data_complete = pd.concat([movie_data_complete, genre_df], axis=1)

print(f"\n✓ Genre encoding complete! Shape: {movie_data_complete.shape}")

# 2.2 MPAA Rating Processing (X8)


In [ ]:
# Step 1: Check current state
print(f"\n1. Before cleaning:")
print(movie_data_complete['rating'].value_counts().head(10))

# Step 2: Clean and standardize
movie_data_complete['rating_clean'] = movie_data_complete['rating'].str.replace('| ', '', regex=False)

# Step 3: Map to standard MPAA (including TV ratings and typos)
rating_mapping = {
    'G': 'G', 'PG': 'PG', 'PG-13': 'PG-13', 'PG--13': 'PG-13',
    'R': 'R', 'NC-17': 'NC-17',
    'Not Rated': 'Not Rated', 'Unrated': 'Not Rated', 'NR': 'Not Rated', 'Unknown': 'Not Rated',
    # Map TV/other ratings to Not Rated
    'TV-MA': 'Not Rated', 'TV-14': 'Not Rated', 'TV-PG': 'Not Rated', 'TV-G': 'Not Rated',
    'Approved': 'Not Rated', 'Open': 'Not Rated', 'MA-17': 'Not Rated'
}

movie_data_complete['rating_clean'] = movie_data_complete['rating_clean'].map(
    lambda x: rating_mapping.get(x, 'Not Rated')
)

print(f"\n2. After standardization (MPAA only):")
print(movie_data_complete['rating_clean'].value_counts())

movie_data_complete = movie_data_complete.loc[:, ~movie_data_complete.columns.duplicated()]
print(f"After deduplication: {movie_data_complete.shape[1]} columns")

# One-Hot vs Ordinal Encoding

For tree-based models (Random Forest, XGBoost, LightGBM), we recommend ONE-HOT ENCODING.

Reasoning:
1. **Tree-based models handle categorical features well**: They can split on any value,
   so ordinal encoding creates an artificial order (G < PG < PG-13 < R < NC-17).

2. **MPAA ratings are NOT strictly ordinal for box office prediction**:
   - While ratings indicate restrictiveness (age restrictions), this doesn't translate 
     to a linear relationship with box office revenue.
   - R-rated movies can outperform PG-13 movies (e.g., R-rated blockbusters like 
     Deadpool, Joker), so the relationship is not monotonic.
   - G and PG ratings might perform differently despite being adjacent in restrictiveness.

3. **One-hot encoding preserves flexibility**:
   - Allows the model to learn independent relationships for each rating category.
   - Tree models can easily combine categories if needed (e.g., "G or PG" splits).
   - No risk of imposing incorrect ordinal assumptions.

4. **Dimensionality is not a concern**:
   - Only 6 categories (G, PG, PG-13, R, NC-17, Not Rated) → only 5 dummy columns.
   - Tree-based models are not affected by multicollinearity like linear models.

CONCLUSION: We'll use ONE-HOT ENCODING (pd.get_dummies) for MPAA ratings.

In [ ]:
# Step 3: Apply one-hot encoding
rating_encoded = pd.get_dummies(
    movie_data_complete['rating_clean'],
    prefix='rating',
    drop_first=False  # Keep all categories for interpretability
)

print(f"\n3. One-hot encoding applied:")
print("-" * 80)
print(f"Created {rating_encoded.shape[1]} binary rating columns:")
print(rating_encoded.columns.tolist())
print(f"\nRating distribution:")
print(rating_encoded.sum().sort_values(ascending=False))

# Step 4: Merge back to main dataframe
movie_data_complete = pd.concat([movie_data_complete, rating_encoded], axis=1)

print(f"\n✓ Rating encoding complete! Shape: {movie_data_complete.shape}")

# 2.3 Verification

In [ ]:
genre_cols = [col for col in movie_data_complete.columns if col.startswith('genre_')]
rating_cols = [col for col in movie_data_complete.columns if col.startswith('rating_')]

print(f"\n✓ Genre features: {len(genre_cols)} binary columns")
print(f"✓ Rating features: {len(rating_cols)} binary columns")
print(f"✓ Total columns: {movie_data_complete.shape[1]}")
print(f"\n✓ Both features are encoded and ready for modeling!")

# Show sample
print("\n" + "-" * 80)
print("SAMPLE: Original vs Encoded Features")
print("-" * 80)
sample_cols = ['genre', 'rating_clean'] + genre_cols[:3] + rating_cols[:3]
print(movie_data_complete[sample_cols].head(5))

### Reflection: Genre and Rating Feature Engineering

**Genre Multi-Label Encoding (X4):**  
We split the comma-separated `genre` column into lists and applied `MultiLabelBinarizer`, creating 32 binary genre columns. This treats each genre independently, allowing movies to have multiple genres simultaneously (e.g., "Action,Adventure,Sci-Fi"). For tree-based models, this approach enables learning both individual genre effects and genre combination patterns that drive box office performance.

**MPAA Rating One-Hot Encoding (X8):**  
We standardized the `rating` column to official MPAA categories (G, PG, PG-13, R, NC-17, Not Rated) by:
- Removing the "| " prefix
- Mapping TV ratings (TV-MA, TV-14, etc.) to "Not Rated"
- Fixing typos (PG--13 → PG-13)
- Consolidating all non-standard values to "Not Rated"

Then applied one-hot encoding to create 6 binary rating columns.

**Why One-Hot Encoding for Rating:**  
We used one-hot encoding rather than ordinal because:
- MPAA restrictiveness doesn't correlate linearly with box office (R-rated *Deadpool* earned $783M, outperforming many PG-13 films)
- Tree-based models learn non-monotonic relationships better with categorical encoding
- Only 6 rating categories means minimal dimensionality impact

**Limitation:** 83% of ratings are "Not Rated" due to missing values, which reduces this feature's predictive power. The `rating_missing` indicator flag helps the model distinguish truly missing ratings from known values.

## 2.4 Seasonality Features (X5 - Release Date)

In [ ]:
# release_month and release_year already extracted in section 1.2.3
print(f"\n1. Release date features already extracted:")
print(f"   - release_month: {movie_data_complete['release_month'].nunique()} unique months")
print(f"   - release_year: {movie_data_complete['release_year'].nunique()} unique years")

# Step 2: Create seasonal categories
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:  # 9, 10, 11
        return 'Fall'

movie_data_complete['season'] = movie_data_complete['release_month'].apply(get_season)

print(f"\n2. Created season categories:")
print(movie_data_complete['season'].value_counts())

# Step 3: Create holiday/peak season indicators
movie_data_complete['summer_release'] = (movie_data_complete['release_month'].isin([5, 6, 7, 8])).astype(int)
movie_data_complete['holiday_release'] = (movie_data_complete['release_month'].isin([11, 12])).astype(int)

print(f"\n3. Created peak season indicators:")
print(f"   - Summer releases (May-Aug): {movie_data_complete['summer_release'].sum()}")
print(f"   - Holiday releases (Nov-Dec): {movie_data_complete['holiday_release'].sum()}")

# Step 4: One-hot encode season (alternative to month)
season_encoded = pd.get_dummies(movie_data_complete['season'], prefix='season')
movie_data_complete = pd.concat([movie_data_complete, season_encoded], axis=1)

print(f"\n4. One-hot encoded seasons:")
print(f"   Created {season_encoded.shape[1]} season columns: {season_encoded.columns.tolist()}")

print(f"\n✓ Seasonality features complete! Shape: {movie_data_complete.shape}")

## 2.5 Production Budget Transformation (X3)

Like box office revenue, production budgets are heavily right-skewed (most films are low-budget; a few blockbusters cost $200M+). We apply the same log transformation for consistency.

In [ ]:
# Analyze seasonality impact on box office

# Group by season
season_stats = movie_data_complete.groupby('season')['worldwide_box_office'].agg([
    'count', 'mean', 'median'
]).sort_values('mean', ascending=False)

print(f"\nBox office by season:")
print(season_stats)

# Group by month
month_stats = movie_data_complete.groupby('release_month')['worldwide_box_office'].agg([
    'count', 'mean', 'median'
]).sort_values('mean', ascending=False)

print(f"\nTop 5 months by average box office:")
print(month_stats.head(5))

# Compare peak vs non-peak
print(f"\nPeak season comparison:")
print(f"Summer releases - Mean: ${movie_data_complete[movie_data_complete['summer_release']==1]['worldwide_box_office'].mean():,.0f}")
print(f"Non-summer releases - Mean: ${movie_data_complete[movie_data_complete['summer_release']==0]['worldwide_box_office'].mean():,.0f}")
print(f"\nHoliday releases - Mean: ${movie_data_complete[movie_data_complete['holiday_release']==1]['worldwide_box_office'].mean():,.0f}")
print(f"Non-holiday releases - Mean: ${movie_data_complete[movie_data_complete['holiday_release']==0]['worldwide_box_office'].mean():,.0f}")

# Log-transform production budget
movie_data_complete['production_budget_log'] = np.log1p(movie_data_complete['production_budget'])

print(f"Production budget - Original range: ${movie_data_complete['production_budget'].min():,.0f} to ${movie_data_complete['production_budget'].max():,.0f}")
print(f"Production budget - Log range: {movie_data_complete['production_budget_log'].min():.2f} to {movie_data_complete['production_budget_log'].max():.2f}")

### Reflection: Seasonality Feature Engineering

**Features Created from Release Date (X5):**  
We extracted temporal features from the `releasedate` column to capture seasonal patterns in box office performance:

1. **Basic temporal features** (already created in section 1.2.3):
   - `release_month`: Numeric month (1-12)
   - `release_year`: Year of release (for inflation/market growth trends)

2. **Season categories**:
   - Created `season` column: Winter, Spring, Summer, Fall
   - One-hot encoded to 4 binary columns: `season_Winter`, `season_Spring`, `season_Summer`, `season_Fall`

3. **Peak season indicators**:
   - `summer_release`: Binary flag for May-August releases (blockbuster season)
   - `holiday_release`: Binary flag for November-December releases (holiday season)

**Why These Features Matter:**  
The movie industry shows strong seasonality:
- **Summer (May-Aug)**: Studios release big-budget blockbusters targeting families and teens on vacation
- **Holiday (Nov-Dec)**: Award contenders and family films capitalize on Thanksgiving/Christmas breaks
- **Dump months (Jan-Feb, Aug-Sep)**: Lower-budget films with weaker box office potential

From the EDA in section 1.3, we observed peaks in summer and winter months, confirming this pattern.

**Encoding Strategy:**  
We provide multiple representations for flexibility:
- `release_month` (numeric): Captures month-specific trends
- `season_*` (one-hot): Groups months into broader seasonal patterns
- `summer_release`, `holiday_release` (binary): Direct indicators for peak release windows

Tree-based models can choose which representation works best for splits. Keeping `release_year` as numeric (not one-hot) preserves the temporal trend (inflation, market growth over time).

## 2.5 Target Variable: Sales Tiers (Classification)

In [ ]:
from scipy.stats import skew

# Step 1: Examine box office distribution and skewness
print(f"\n1. Worldwide box office distribution:")
print(f"   Count: {movie_data_complete['worldwide_box_office'].count()}")
print(f"   Mean: ${movie_data_complete['worldwide_box_office'].mean():,.0f}")
print(f"   Median: ${movie_data_complete['worldwide_box_office'].median():,.0f}")
print(f"   Min: ${movie_data_complete['worldwide_box_office'].min():,.0f}")
print(f"   Max: ${movie_data_complete['worldwide_box_office'].max():,.0f}")
print(f"   Std: ${movie_data_complete['worldwide_box_office'].std():,.0f}")

# Check skewness
skewness = skew(movie_data_complete['worldwide_box_office'].dropna())
print(f"   Skewness: {skewness:.2f} (heavily right-skewed if > 1)")

# Step 2: Use LOG-TRANSFORMED data for tier boundaries
# This handles outliers better and creates more meaningful business segments
print(f"\n2. Creating tiers using LOG-TRANSFORMED box office:")
print(f"   Rationale: Log transformation reduces impact of extreme outliers")
print(f"              and creates more balanced tier separations")

# Calculate tertiles on LOG scale
log_q33 = movie_data_complete['worldwide_box_office_log'].quantile(0.33)
log_q67 = movie_data_complete['worldwide_box_office_log'].quantile(0.67)

# Convert back to original scale for interpretability
q33_actual = np.expm1(log_q33)  # inverse of log1p
q67_actual = np.expm1(log_q67)

print(f"\n3. Sales tier boundaries (based on log tertiles):")
print(f"   Low tier:    < ${q33_actual:,.0f} (bottom 33%)")
print(f"   Medium tier: ${q33_actual:,.0f} - ${q67_actual:,.0f} (middle 34%)")
print(f"   High tier:   > ${q67_actual:,.0f} (top 33%)")

# Step 3: Create sales tier using log-based boundaries
def classify_sales_log(box_office_log):
    if box_office_log < log_q33:
        return 'Low'
    elif box_office_log < log_q67:
        return 'Medium'
    else:
        return 'High'

movie_data_complete['sales_tier'] = movie_data_complete['worldwide_box_office_log'].apply(classify_sales_log)

print(f"\n4. Sales tier distribution:")
tier_counts = movie_data_complete['sales_tier'].value_counts().sort_index()
print(tier_counts)
print(f"\n   Balance check: {tier_counts.min()}/{tier_counts.max()} = {tier_counts.min()/tier_counts.max():.2%} (should be ~100%)")

# Step 5: Show actual box office ranges per tier
print(f"\n5. Actual box office ranges per tier (original scale):")
for tier in ['Low', 'Medium', 'High']:
    tier_data = movie_data_complete[movie_data_complete['sales_tier'] == tier]['worldwide_box_office']
    print(f"   {tier:7s}: ${tier_data.min():>12,.0f} - ${tier_data.max():>12,.0f} (median: ${tier_data.median():>12,.0f})")

# Step 6: Create numeric encoding for target (for some models)
tier_mapping = {'Low': 0, 'Medium': 1, 'High': 2}
movie_data_complete['sales_tier_encoded'] = movie_data_complete['sales_tier'].map(tier_mapping)

print(f"\n6. Numeric encoding created: Low=0, Medium=1, High=2")
print(f"\n✓ Target variable created! Shape: {movie_data_complete.shape}")

In [ ]:
# Descriptive statistics by tier
tier_stats = movie_data_complete.groupby('sales_tier')['worldwide_box_office'].agg([
    'count', 'mean', 'median', 'min', 'max'
])
print(f"\nBox office statistics by tier:")
print(tier_stats)

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution of sales tiers
tier_counts = movie_data_complete['sales_tier'].value_counts().sort_index()
axes[0].bar(tier_counts.index, tier_counts.values, color=['#d62728', '#ff7f0e', '#2ca02c'])
axes[0].set_title('Sales Tier Distribution', fontsize=14)
axes[0].set_xlabel('Sales Tier')
axes[0].set_ylabel('Number of Movies')
for i, v in enumerate(tier_counts.values):
    axes[0].text(i, v + 100, str(v), ha='center', va='bottom')

# Box office by tier (log scale for visibility)
movie_data_complete.boxplot(column='worldwide_box_office_log', by='sales_tier', 
                            ax=axes[1], patch_artist=True)
axes[1].set_title('Box Office Distribution by Tier (log scale)', fontsize=14)
axes[1].set_xlabel('Sales Tier')
axes[1].set_ylabel('Log(Worldwide Box Office)')
plt.suptitle('')  # Remove auto-title

plt.tight_layout()
plt.show()

print(f"\n✓ Sales tier analysis complete!")

### Reflection: Target Variable Creation

**Sales Tier Classification:**  
Based on the research questions (sections 0 and 0.1), this project aims to predict whether a movie will achieve **low, medium, or high sales**. We created the classification target variable `sales_tier` by splitting `worldwide_box_office` into tertiles **using log-transformed values**.

**Why Log-Based Tertiles:**  
The EDA (section 1.3) revealed that `worldwide_box_office` is **heavily right-skewed** (most movies earn modest amounts, few earn billions). Standard tertiles on raw values would:
- Place the boundary between Low/Medium at a very low value (e.g., $2-3M)
- Create a "High" tier dominated by extreme outliers (blockbusters earning >$500M)
- Result in poor separation between tiers for meaningful business decisions

Using **log-transformed tertiles** addresses this:
1. **Reduces outlier impact**: Log transformation compresses the scale, preventing blockbusters from distorting boundaries
2. **Creates meaningful segments**: Boundaries reflect proportional differences in performance, not absolute dollar gaps
3. **Balanced classes**: Ensures ~33% of movies in each tier
4. **Aligns with business reality**: Movie industry thinks in terms of magnitude (e.g., "10x return"), not absolute differences

**Tier Definitions (Log-Based):**
- **Low tier** (0): Bottom 33% on log scale - Movies underperforming relative to the distribution
- **Medium tier** (1): Middle 34% on log scale - Moderately successful movies
- **High tier** (2): Top 33% on log scale - Strong commercial performers and blockbusters


**Statistical Validity:**
- **Skewness handled**: Log transformation normalizes distribution before splitting
- **Class balance**: Tertiles ensure balanced classes for unbiased model training
- **Interpretability**: Converting boundaries back to original scale shows actual dollar ranges
- **Robustness**: Log-based approach is standard for skewed financial data (box office, revenue, sales)

**Target Variables Created:**
1. `sales_tier`: Categorical (Low/Medium/High) - primary target for classification
2. `sales_tier_encoded`: Numeric (0/1/2) - for models requiring numeric targets
3. `worldwide_box_office`: Continuous - kept for reference
4. `worldwide_box_office_log`: Log-transformed - used for tier creation

**Research Alignment:**  
This approach directly supports:
- **Business question**: Predict performance tiers to allocate marketing budget efficiently
- **Subquestion 1**: Analyze feature relationships with sales tiers
- **Subquestion 2**: Train ML models to predict tier membership
- **Subquestion 4**: SHAP analysis of high vs low sales drivers
- **Subquestion 5**: Identify market segments (clusters) associated with each tier

**Class Balance Verification:**  
The log-based tertile approach ensures approximately equal class sizes (~33% each), which is crucial for:
- Avoiding model bias toward majority class
- Accurate evaluation metrics (precision, recall, F1-score)
- Fair SHAP value comparisons across tiers
- Meaningful feature importance interpretation

## 2.7 Feature Engineering Summary

### Final Feature Set for Modeling

| Category | Features | Count | Notes |
|----------|----------|-------|-------|
| **Target** | `sales_tier_encoded` | 1 | 0=Low, 1=Medium, 2=High |
| **Numeric** | `metascore`, `userscore`, `runtime`, `production_budget_log`, `release_year` | 5 | Continuous features |
| **Genre** | `genre_*` | 32 | Binary (multi-label encoded) |
| **Rating** | `rating_*` | 6 | Binary (one-hot encoded) |
| **Seasonality** | `season_*`, `summer_release`, `holiday_release` | 6 | Binary indicators |

**Total features for modeling:** ~49 (excluding intermediate/raw columns)

### Features Excluded from Modeling
- `worldwide_box_office`, `worldwide_box_office_log`: Target variable (used to create tiers)
- `opening_weekend`, `theatre_count`, `avg_run_per_theatre`: Data leakage risk
- `genre`, `rating`, `rating_clean`, `season`: Raw categorical columns (encoded versions used instead)
- `title`, `url`, `movie_id`: Identifiers, not predictive

### Next Steps
1. **Train/Validation/Test Split** - Stratified by `sales_tier`
2. **Feature Scaling** - StandardScaler for numeric features (fit on train only)
3. **Handle remaining missing values** - Imputation strategy for `runtime`, `userscore`